In [1]:
import pandas as pd
import requests
import lxml.html
import re
from tqdm import tqdm
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup 
import json

D:\anaconda\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [22]:
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 OPR/82.0.4227.50'}
r = requests.get('https://www.brw.pl/api/cms/view?uri=%2Fsiec-sprzedazy%2F&id_projektu=4&lang=pl&sciezka=siec-sprzedazy&id_kategorii=19&symbol=BrwSalony', headers=headers).json()
print(r)


{'get': {'uri': '/siec-sprzedazy/', 'id_projektu': '4', 'lang': 'pl', 'sciezka': 'siec-sprzedazy', 'id_kategorii': '19', 'symbol': 'BrwSalony', 'wpis': ''}, 'result': {'wojewodztwa': {'': 'Wybierz województwo', '1': 'dolnośląskie', '2': 'kujawsko-pomorskie', '3': 'lubelskie', '4': 'lubuskie', '5': 'łódzkie', '6': 'małopolskie', '7': 'mazowieckie', '8': 'opolskie', '9': 'podkarpackie', '10': 'podlaskie', '11': 'pomorskie', '12': 'śląskie', '13': 'świętokrzyskie', '14': 'warmińsko-mazurskie', '15': 'wielkopolskie', '16': 'zachodniopomorskie'}, 'panstwa': None, 'script': '<script src="/szablony/glowny/scripts/leaflet.js" type="text/javascript"></script>\n<script src="/szablony/glowny/scripts/salony_osm.min.js?v=1.3" type="text/javascript"></script>\n<script type="text/javascript">\n                //<![CDATA[\n                var ajax_adres = "https://www.brw.pl/ajax/w4_TtVytqKl02c2S0sNczJqkhuuQk9A/";\n                var ajax_punkty = "https://www.brw.pl/ajax/zpLIv5maeKSYy8KP07immqanj-PV

In [23]:
r.keys()

dict_keys(['get', 'result', 'seo', 'cfg'])

In [25]:
len(r['result']['lista_salonow'])

437

In [27]:
r['result']['lista_salonow'][2]

{'id': 551,
 'id_projektu': 2,
 'lang': 'pl',
 'maat_id': '1000/PRUSZCZ GD',
 'data_dodania': '2012-08-07',
 'nazwa_salonu': 'Salon Firmowy Black Red White',
 'opis': '<style type="text/css">\r\n.opsalonu { font-size:14px; font-weight:normal; line-height:25px; text-align: justify;}\r\n.do-pobrania { padding-left: 20px; background: url(/szablony/glowny/images/ico-arr-right.png) no-repeat left center; }</style>\r\n<div class="opsalonu">\r\n\t<p>\r\n\t\t<strong>PLIKI DO POBRANIA</strong></p>\r\n\t<div class="do-pobrania">\r\n\t\tOgólne warunki świadczenia usług transportowych oraz wniesienia&nbsp;- <u><a href="/gfx/brw/userfiles/_public/pliki_do_pobrania/Ogolne_warunki_swiadczenia_uslug_transportowych_oraz_wniesienia.pdf" target="_blank">pobierz</a></u></div>\r\n\t<div class="do-pobrania">\r\n\t\tOgólne warunki realizacji usług montażu mebli oraz sprzętu AGD&nbsp;- <u><a href="/gfx/brw/userfiles/_public/pliki_do_pobrania/Ogolne_warunki_realizacji_uslug_montazu_mebli_oraz_sprzetu_AGD.pdf" 

In [28]:
lista = r['result']['lista_salonow']

In [30]:
data = {'nazwa':[],
        'brand':[],
        'pelny_adres':[],
       'ulica':[],
       'kod_pocztowy':[],
       'miasto':[],
       'lat':[],
       'lng':[]}

for s in lista:
    data['nazwa'].append('Black Red White')
    data['brand'].append('Black Red White')
    data['ulica'].append(s['adres'])
    data['miasto'].append(s['miejscowosc'])
    data['kod_pocztowy'].append(s['kod_pocztowy'])
    data['lat'].append(str(s['mapa_lat']))
    data['lng'].append(str(s['mapa_lng']))
    data['pelny_adres'].append(str(s['adres'])+' '+str(s['kod_pocztowy'])+' '+str(s['miejscowosc']))
    
df = pd.DataFrame(data)
df

,nazwa,brand,pelny_adres,ulica,kod_pocztowy,miasto,lat,lng
0,Black Red White,Black Red White,"ul. Wojska Polskiego 3, H-13 39-300 Mielec","ul. Wojska Polskiego 3, H-13",39-300,Mielec,50.3109508640716,21.4643561840052
1,Black Red White,Black Red White,ul. Czekoladowa 24 55-040 Wrocław,ul. Czekoladowa 24,55-040,Wrocław,51.047824,16.9654041
2,Black Red White,Black Red White,ul. Grunwaldzka 115 83-000 Pruszcz Gdański,ul. Grunwaldzka 115,83-000,Pruszcz Gdański,54.2748712,18.6385573
3,Black Red White,Black Red White,ul. Popularna 75 (obok Castoramy) 02-226 Warszawa,ul. Popularna 75 (obok Castoramy),02-226,Warszawa,52.2025570518569,20.9359073638916
4,Black Red White,Black Red White,ul. Szkolna 4 24-320 Poniatowa,ul. Szkolna 4,24-320,Poniatowa,51.190491,22.06436
...,...,...,...,...,...,...,...,...
432,Black Red White,Black Red White,ul. Młodzieżowa 54 09-100 Płońsk,ul. Młodzieżowa 54,09-100,Płońsk,52.6361390157333,20.3574456830811
433,Black Red White,Black Red White,ul. Tunelowa 2 38-100 Strzyżów,ul. Tunelowa 2,38-100,Strzyżów,49.8700775,21.8012433
434,Black Red White,Black Red White,ul. Podwale 7 68-200 Żary,ul. Podwale 7,68-200,Żary,51.6372606,15.1426227
435,Black Red White,Black Red White,ul. Kupiecka 1A 66-200 Świebodzin,ul. Kupiecka 1A,66-200,Świebodzin,52.2477347,15.5381744


In [31]:
df['ulica'] = [x.replace('ul. ', '') for x in df['ulica']]
df['ulica'] = [x.replace('al. ', '') for x in df['ulica']]
df['ulica'] = [x.replace('Ul. ', '') for x in df['ulica']]
df['ulica'] = [x.replace('Al. ', '') for x in df['ulica']]
df['ulica'] = [x.replace('Ul.', '') for x in df['ulica']]
df['ulica'] = [x.replace('Al.', '') for x in df['ulica']]
df['ulica'] = [x.replace('ul.', '') for x in df['ulica']]
df['ulica'] = [x.replace('al.', '') for x in df['ulica']]
df['ulica'] = [x.replace('pl.', 'plac') for x in df['ulica']]
df

,nazwa,brand,pelny_adres,ulica,kod_pocztowy,miasto,lat,lng
0,Black Red White,Black Red White,"ul. Wojska Polskiego 3, H-13 39-300 Mielec","Wojska Polskiego 3, H-13",39-300,Mielec,50.3109508640716,21.4643561840052
1,Black Red White,Black Red White,ul. Czekoladowa 24 55-040 Wrocław,Czekoladowa 24,55-040,Wrocław,51.047824,16.9654041
2,Black Red White,Black Red White,ul. Grunwaldzka 115 83-000 Pruszcz Gdański,Grunwaldzka 115,83-000,Pruszcz Gdański,54.2748712,18.6385573
3,Black Red White,Black Red White,ul. Popularna 75 (obok Castoramy) 02-226 Warszawa,Popularna 75 (obok Castoramy),02-226,Warszawa,52.2025570518569,20.9359073638916
4,Black Red White,Black Red White,ul. Szkolna 4 24-320 Poniatowa,Szkolna 4,24-320,Poniatowa,51.190491,22.06436
...,...,...,...,...,...,...,...,...
432,Black Red White,Black Red White,ul. Młodzieżowa 54 09-100 Płońsk,Młodzieżowa 54,09-100,Płońsk,52.6361390157333,20.3574456830811
433,Black Red White,Black Red White,ul. Tunelowa 2 38-100 Strzyżów,Tunelowa 2,38-100,Strzyżów,49.8700775,21.8012433
434,Black Red White,Black Red White,ul. Podwale 7 68-200 Żary,Podwale 7,68-200,Żary,51.6372606,15.1426227
435,Black Red White,Black Red White,ul. Kupiecka 1A 66-200 Świebodzin,Kupiecka 1A,66-200,Świebodzin,52.2477347,15.5381744


In [32]:
df.to_excel('blackredwhite.xlsx')